In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from matplotlib import colormaps as cm
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path
from statsmodels.nonparametric.smoothers_lowess import lowess

# Ignore FutureWarnings from geopandas
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', message='.*initial implementation of Parquet.*')
# Ignore SettingWithCopyWarning from (geo-)pandas
pd.options.mode.chained_assignment = None  # default='warn'


# Config
day = 170

in_dir = Path(f'../../export/{day}')
out_dir = Path(in_dir / 'sensitivity_analysis')
out_dir.mkdir(parents=True, exist_ok=True)

tod_list = ['10am', '1pm', '4pm', '7pm']
sensitivity_factors = [0.2, 0.4, 0.6, 0.8, 1.0]
len_types = ['short', 'long']
optimized_type = 'shaded'

sns.set_theme(style='whitegrid')
sns.set_context('paper')

xlim = (-2, 102)
ylim = (-2, 102)

In [2]:
# Helper functions
def tod_format(tod):
    """Returns the formatted time of day string"""
    return tod[:-2] + ' ' + tod[-2:].upper()

# Function to slice a colormap
def slice_cmap(cmap, min_val=0.0, max_val=1.0):
    """Returns a sliced colormap"""
    return LinearSegmentedColormap.from_list(
        f'sliced_{cmap.name}',
        cmap(np.linspace(min_val, max_val, 256))
    )

## Sensitivity Factor Analysis

Preprocessing

In [ ]:
results = pd.DataFrame()
for sensitivity_factor in sensitivity_factors:
    data_dir = in_dir / f'{sensitivity_factor}' / 'exportdata'
    data = gpd.read_feather(data_dir / 'all_routes_statistics.feather')

    data['sensitivity_factor'] = sensitivity_factor

    results = pd.concat([results, data], ignore_index=True)

In [4]:
results.to_feather(out_dir / 'all_routes_statistics_170_sensitivities.feather', index=False)

Start from here

In [5]:
results = pd.read_feather(out_dir / 'all_routes_statistics_170_sensitivities.feather')
results_ex = results[results['len_diff_rel'] > 0]

In [ ]:
results_ex.groupby('sensitivity_factor')['sol_expo_reduction'].mean()

In [ ]:
results_ex['len_type'] = np.where(results_ex['distance'] < 1000, 'short', 'long')

## Smoother comparison: sensitivities & short/long

In [ ]:
x_value = 'len_diff_rel'
y_value = 'sol_expo_reduction'

norm = plt.Normalize(0, max(sensitivity_factors))
cmap = cm.get_cmap('plasma_r')

for len_type in len_types:
    fig, axs = plt.subplots(1, len(tod_list), figsize=(17, 5))

    df_lt = results_ex[results_ex['len_type'] == len_type]
    for col, tod in enumerate(tod_list):
        ax = axs[col]
        df_lt_tod = df_lt[df_lt['route_type'] == f'{optimized_type} route at {tod}']
        ax.plot((0, 100), (0, 100), color='silver', linestyle=':', linewidth=1)
        for sensitivity_factor in sensitivity_factors:
            df_lt_sw = df_lt_tod[df_lt_tod['sensitivity_factor'] == sensitivity_factor]
            xmean_heal = df_lt_sw[x_value].mean()
            ymean_heal = df_lt_sw[y_value].mean()
            # Compute a lowess smoothing of the data
            smoothed = lowess(exog=df_lt_sw[y_value], endog=df_lt_sw[x_value], frac=0.2)
            color = cmap(norm(sensitivity_factor))
            ax.plot(smoothed[:, 1], smoothed[:, 0], color=color, linestyle='-', linewidth=2, label=sensitivity_factor)
            ax.axhline(ymean_heal, color=color, linestyle='--', linewidth=1)
            ax.axvline(xmean_heal, color=color, linestyle='--', linewidth=1)

        ax.set_title(tod_format(tod), fontsize=16)
        ax.set_xlim(xlim)
        ax.set_xlabel('Length increase [%]', fontsize=16)
        ax.set_ylim(ylim)
        ax.set_ylabel('Solar exposure reduction [%]', fontsize=16) if col == 0 else ax.set_ylabel('')
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=12)
    ax.legend(title='Sensitivity\nfactor', title_fontsize=12, fontsize=12, loc='upper right')

    plt.tight_layout()
    plt.savefig(out_dir / f'lowess_sensitivities_{len_type}.png', dpi=300, bbox_inches='tight')
    plt.show()